In [ ]:
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import matplotlib
font = {
        'weight' : 'bold',
        'size'   : 27}

matplotlib.rc('font', **font)
import h5py
import plotly.graph_objs as go
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
import pandas as pd
import sys
import os
import xarray as xr

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [2]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [3]:
def divergence(f,grid:tuple):
        num_dims = len(f)
        result = np.ufunc.reduce(np.add, [np.gradient(f[i], grid[i], axis=i) for i in range(num_dims)])
       
        return result


In [4]:
a=5/3
path_save = rf"H:\phd stuff\tidy3d\output\Divergence 3D\Test"

In [ ]:
values = []
for path_direction in [
                  #      fr"H:\phd stuff\tidy3d\data\08_30_2024 Frequency Monitor"
                      #  fr"H:\phd stuff\tidy3d\data\09_03_2024 Freq Monitors Analysis Divergence\z_incidence",
                       fr"H:\phd stuff\tidy3d\data\09_05_2024 Freq Monitors Analysis Divergence\z_incidence"

                      
                       ]:

      folder_path = f"{path_direction}"


      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            

            frames = []
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                field_array = structure_1.sim_data["freq_monitorField"]
                field_freq_intensity = structure_1.sim_data.get_intensity("freq_monitorField")[:,:,int(np.shape(field_array.Ex.z)[0]/2),:]
                # del structure_1
                monitor_lambdas=np.array([0.35,0.5]) 
                AM.create_movie(field_freq_intensity,monitor_lambdas,name=f"freqmonitor {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}",type="f",log=False,path=fr"H:\phd stuff\tidy3d\output/field3D/{filename}",frames_per_second=1,normalize=False,a=a)

                min_div = []
                max_div = []
                mean_div = []
                abs_mean = []
                #Calculate div#################################################
                for index in range(np.shape(field_array.Ex)[-1]):
                  norm = np.linalg.norm(np.array([field_array.Ex[:,:,:,index].values,field_array.Ey[:,:,:,index].values,field_array.Ez[:,:,:,index].values]))
                  F = np.array([field_array.Ex[:,:,:,index].values/norm,field_array.Ey[:,:,:,index].values/norm,field_array.Ez[:,:,:,index].values/norm])
                  result = divergence(F,(field_array.Ex.x,field_array.Ex.y,field_array.Ex.z))
                  div = xr.DataArray(result, coords={"x":field_array.Ex.x/a,"y":field_array.Ex.y/a,"z":field_array.Ex.z/a}, dims=["x", "y", "z"])
                  # fig, ax = plt.subplots(figsize=(14,10))
                #pcolormesh at z=0
                  # pcolormesh = np.real(div).isel(z=int(np.shape(field_array.Ex.z)[0]/2)).plot.pcolormesh(ax=ax,cmap="RdBu")
                  # plt.title(f'$\\nu$: {a/(td.C_0/field_array.Ex.f[index]):.4g}')
                  # plt.xlim(-9,9)
                  # plt.ylim(-9,9)
                  # plt.ylabel("Y")
                  # plt.xlabel("X")
                  # cbar = pcolormesh.colorbar
                  # cbar.set_label(rf"$Re(\nabla \cdot E)$")  # Replace with your desired label

                  # ax.set_aspect('equal', adjustable='box')
                  # if not os.path.exists(f'{path_save}/{filename}/pics'):
                  #       os.makedirs(f'{path_save}/{filename}/pics')
                  #       print(f"Folder {path_save}/{filename}/pics created successfully.")
                  # plt.savefig(rf'{path_save}/{filename}/pics/frame_{index}.png')
                  # plt.close(fig)
                  # frames.append(f'{path_save}/{filename}/pics/frame_{index}.png')
                  mean_div+=[np.mean((result.flatten()))]
                  abs_mean+=[np.mean(np.abs(result.flatten()))]
                  min_div+=[np.min((result.flatten()))]
                  max_div+=[np.max((result.flatten()))]
                 ###########################################################
            freqs = a/(td.C_0/field_array.Ex.f)
            #Create movie
            # with imageio.get_writer(rf'{path_save}/{filename}.mp4', fps=1) as writer:
            #       for frame in frames:
            #             image = imageio.imread(frame)
            #             writer.append_data(image)
            ############################################################
            try:
              transmission = structure_1.sim_data['flux1'].flux
              transmission0 = structure_1.sim_data0['flux1'].flux
              transmission_normalized = transmission / transmission0
              values +=[
                {
                    "abs_mean":abs_mean,
                    "filename":filename,
                    "transmission":transmission_normalized
                }
            ]
            except:
                values +=[
                {
                    "abs_mean":abs_mean,
                    "filename":filename,
                    "transmission":[]
                }
            ]

            
            
            # plt.figure(figsize=(20, 10))
            # plt.plot(freqs,mean_div,label=fr"Mean($\nabla \cdot E$)")
            # plt.plot(freqs,min_div,label=fr"Min($\nabla \cdot E$)")
            # plt.plot(freqs,max_div,label=fr"Max($\nabla \cdot E$)")
            # plt.title(filename)
            # plt.legend()
            # plt.xlabel(rf"$\nu$")
            # plt.show()

            # plt.figure(figsize=(20, 10))
            # plt.scatter(freqs,mean_div,label=fr"Mean($\nabla \cdot E$)")
            # plt.title(filename)
            # plt.legend()
            # plt.xlabel(rf"$\nu$")
            # plt.show()

            # plt.figure(figsize=(20, 10))
            # plt.scatter(freqs,abs_mean,label=fr"Mean($|\nabla \cdot E|$)")
            # plt.title(filename)
            # plt.legend()
            # plt.grid()
            # plt.xlabel(rf"$\nu$")
            # plt.show()

            # try:
            #   plt.figure(figsize=(20,10))
              
            #   plt.plot(freqs,transmission_normalized, label=f"Transmission")
            #   plt.xlabel(r'$\nu$')
            #   plt.ylabel('Transmission [%]')
            #   plt.legend()
            #   plt.grid()
            #   plt.yscale("log")
            #   plt.show()
            # except:
            #     continue
        


     

               



In [ ]:
for item in values: 
    print(item["filename"])

np.shape(values)

In [ ]:
# Create some data
x = freqs
# Create the first plot
fig, ax1 = plt.subplots(figsize=(20, 10))
names = ["RCP","Dielectric Spheres","PEC Spheres"]
for i,item in enumerate(values):
    if i>1:
        ax1.plot(x,values[i]["abs_mean"], label=names[i-2]) # First plot in green
ax1.set_xlabel(rf'$\nu$')
ax1.set_ylabel(fr"Mean($|\nabla \cdot E|$)", color='black')
ax1.tick_params(axis='y', labelcolor='black')


# Create a secondary y-axis that shares the same x-axis
# ax2 = ax1.twinx()
# ax2.plot(x, values[2]["transmission"],label="Transmission", color="#000305")  # Second plot in blue
# ax2.set_ylabel('Transmission[%]', color='#000305')
# ax2.set_yscale("log")
# ax2.tick_params(axis='y', labelcolor='#000305')

# Show both plots on the same figure
fig.tight_layout()
fig.legend( bbox_to_anchor=(0.9, 0.9))
plt.title("Mean Divergence")
plt.show()

In [ ]:
td.C_0/(a/0.42)